# Spark
Please upload provided files: sessions.json, camapigns.json, and orders.json

In [ ]:
# The following is code for uploading a file to the colab.research.google 
# environment.

# library for uploading files
from google.colab import files 

def upload_files():
    # initiates the upload - follow the dialogues that appear
    uploaded = files.upload()

    # verify the upload
    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])))

    # uploaded files need to be written to file to interact with them
    # as part of a file system
    for filename in uploaded.keys():
        with open(filename, 'wb') as f:
            f.write(uploaded[filename])

In [ ]:
# Upload sessions.json, camapigns.json, and orders.json
# to a notebook by calling the function above
upload_files()

Saving campaigns.json to campaigns.json
Saving orders.json to orders.json
Saving sessions.json to sessions.json
User uploaded file "campaigns.json" with length 828520 bytes
User uploaded file "orders.json" with length 296814 bytes
User uploaded file "sessions.json" with length 2515540 bytes


Initialize spark enviroment:

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()

# Initialize data frames (1 point)

Please start Spark session and load three datasets (and set aliases to sessions, orders, campaigns).

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local") \
.appName("App Name") \
.config("spark.some.config.option", "some-value") \
.getOrCreate()


In [26]:
df_campaigns = spark.read.json('campaigns.json')
df_campaigns.alias('campaigns')
df_orders = spark.read.json('orders.json')
df_orders.alias('orders')
df_sessions = spark.read.json('sessions.json')
df_sessions.alias('sessions')

DataFrame[browser: string, os: string, session_id: string, session_start: string]

# Exploratory Analysis (4 points)

*   List item
*   List item



Print schema for each dataframe (1 point)

In [27]:
df_campaigns.printSchema()

root
 |-- campaign: string (nullable = true)
 |-- session_id: string (nullable = true)



In [28]:
df_orders.printSchema()

root
 |-- number_of_items: long (nullable = true)
 |-- session_id: string (nullable = true)
 |-- transaction_value: double (nullable = true)



In [29]:
df_sessions.printSchema()

root
 |-- browser: string (nullable = true)
 |-- os: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- session_start: string (nullable = true)



Show count of sessions by browser, by OS, and by both browser and OS (1 point)

In [32]:
df_sessions.show(15)

+-------+-------+--------------------+-------------------+
|browser|     os|          session_id|      session_start|
+-------+-------+--------------------+-------------------+
|Firefox| Mac OS|99c24019-0d95-472...|2018-04-20 00:00:05|
|Firefox| Mac OS|3155cff6-4b22-4eb...|2018-04-20 00:00:25|
| Chrome|Windows|7f93a680-0634-4e3...|2018-04-20 00:00:27|
|Firefox|Windows|fedaf382-25e3-4dc...|2018-04-20 00:00:29|
|Firefox|Windows|5eeef4e0-fc6c-4ed...|2018-04-20 00:00:49|
| Chrome|Windows|da4cdcd6-4393-4fe...|2018-04-20 00:00:50|
|Firefox|Windows|067cb75e-0051-4cb...|2018-04-20 00:01:11|
|Firefox|Windows|158d5493-34e9-441...|2018-04-20 00:01:13|
|Firefox| Mac OS|c025616a-f5d6-4a8...|2018-04-20 00:01:18|
| Chrome|Windows|f1dc344d-aacd-4c6...|2018-04-20 00:01:39|
|Firefox| Mac OS|63ef9d7e-1334-4cd...|2018-04-20 00:01:41|
|Firefox|Windows|969a8333-1cd9-449...|2018-04-20 00:01:42|
|Firefox|Windows|1a9ab477-5bbd-4d1...|2018-04-20 00:01:45|
| Chrome| Mac OS|85752fc9-7276-4ce...|2018-04-20 00:02:0

In [47]:
df_sessions.groupby('browser').count().show()

+-------+-----+
|browser|count|
+-------+-----+
|Firefox| 6381|
| Safari|  883|
| Chrome| 9999|
|   Edge| 2737|
+-------+-----+



In [48]:
df_sessions.groupby('os').count().show()

+-------+-----+
|     os|count|
+-------+-----+
| Mac OS| 5367|
|Windows|14633|
+-------+-----+



Show count of sessions by day and hour of the day, e.g., 2018-04-20 00:00:05 should become 2018-04-20 00

(hint create new column hour of the day based on session_start, ad this column to sessions dataframe as it will be needed later) (2 points)

In [46]:
df_sessions.select(hour('session_start').alias('hour')).collect()

SyntaxError: ignored

Show the same count as in previous step, but sorted by ascending hour_of_day, **please show all 24 hours** (1 point)

# Preparing aggregated report (extra)
Join sessions with orders (creating new dataframe that will be used in later steps) and add is_transaction column (1 if transaction_value is present, 0 otherwise) 

To validate step display first 20 rows

Join the dataframe created in previous step with campaigns, and create a dataframe that will have count of sessions, total transaction value, total number of sold items, and number of transactions per each day and hour, campaign, browser, and os 

To validate step display first 20 rows